In [ ]:
# default_exp core.rq_marketplace

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *
import json
import responses
from pathlib import Path
from discogspy.core import *

In [ ]:
#hide
with open(Path("../config/example_user_info.json"), "r") as user_config_file:
    config = json.load(user_config_file)
    example_user = UserWithUserTokenBasedAuthentication(user_token=config["user_token"],
                                                        user_agent=config["user_agent"])

# Core Marketplace Requests

> Core.rq_marketplace is a collection of function wrappers around the Discogs api for marketplace interactions.

In [ ]:
#export
import requests
from typing import Union
from discogspy.core import *

## Inventory

In [ ]:
#export


def get_inventory(user: Union[UserWithoutAuthentication,
                              UserWithUserTokenBasedAuthentication], 
                  username: str,
                  status: Union[StatusInventory, None] = None,
                  page: Union[int, None] = None,
                  per_page: Union[int, None] = None,
                  sort: Union[SortOptionsInventory, None] = None,
                  sort_order: Union[SortOrder, None] = None) -> requests.models.Response:
    """
    Get a list of listings of a given user inventory.
    
    No user Authentication needed. 
    
    Note from the Discogs API: "If you are not authenticated as the inventory owner, 
    only items that have a status of For Sale will be visible.
    If you are authenticated as the inventory owner you will get additional weight, 
    format_quantity, external_id, and location keys.
    If the user is authorized, the listing will contain a in_cart boolean 
    field indicating whether or not this listing is in their cart."    
    """
    url = f"{USERS_URL}/{username}/inventory"
    headers = user.headers
    params = user.params
    if status:
        if type(user) is UserWithoutAuthentication:
            params["status"] = StatusInventory.for_sale.value
        else:
            params["status"] = status.value
    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)
    if sort:
        if type(user) is UserWithoutAuthentication and (sort is not SortOptionsInventory.status and 
                                                        sort is not SortOptionsInventory.location):
            params["sort"] = sort.name   
        else:
            params["sort"] = sort.name   
    if sort_order:
        params["sort_order"] = sort_order.name
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_inventory
get_inventory(user=example_user, username="Les_nuits_noires", status=StatusInventory.for_sale,
              page=1, per_page=3, sort=SortOptionsInventory.price, sort_order=SortOrder.asc)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_inventory___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{USERS_URL}/user_name/inventory?status=For+Sale&page=1&per_page=3&sort=price&sort_order=asc"
    
    result_content = {"TestInventory": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_inventory(user=test_user, username="user_name", status=StatusInventory.for_sale,
              page=1, per_page=3, sort=SortOptionsInventory.price, sort_order=SortOrder.asc)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_inventory___authentication_none()

@responses.activate
def test_get_inventory___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{USERS_URL}/user_name/inventory?token=test_user_token&status=Sold&page=1&per_page=3&sort=price&sort_order=asc"
    
    result_content = {"TestInventory": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_inventory(user=test_user, username="user_name", status=StatusInventory.sold,
              page=1, per_page=3, sort=SortOptionsInventory.price, sort_order=SortOrder.asc)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_inventory___authentication_user_token()

## Listing

In [ ]:
#export


def get_listing(user: Union[UserWithoutAuthentication,
                            UserWithUserTokenBasedAuthentication], 
                listing_id: int,
                curr_abbr: Union[CurrAbbr, None] = None) -> requests.models.Response:
    """
    Get all the information about the requested listing from the given user inventory.
    
    No user Authentication needed. 
    
    Note from the Discogs API: "If the authorized user is the listing owner the listing 
    will include the weight, format_quantity, external_id, and location keys. 
    If the user is authorized, the listing will contain a in_cart boolean field 
    indicating whether or not this listing is in their cart."
    """
    url = f"{LISTINGS_URL}/{listing_id}"
    headers = user.headers
    params = user.params
    if curr_abbr: 
        params["curr_abbr"] = curr_abbr.value
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_listing
get_listing(user=example_user, listing_id=1034876519, curr_abbr=CurrAbbr.EUR)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_listing___authentication_none():
    test_user = UserWithoutAuthentication()
    
    target_url = f"{LISTINGS_URL}/123456?curr_abbr=EUR"
    
    result_content = {"TestListing": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_listing(user=test_user, listing_id=123456, curr_abbr=CurrAbbr.EUR)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_listing___authentication_none()

@responses.activate
def test_get_listing___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{LISTINGS_URL}/123456?token=test_user_token&curr_abbr=EUR"
    
    result_content = {"TestListing": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_listing(user=test_user, listing_id=123456, curr_abbr=CurrAbbr.EUR)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_listing___authentication_user_token()

In [ ]:
#hide

from nbdev.export import *
notebook2script()

Converted 00_core.constants.ipynb.
Converted 01_core.discogs_user.ipynb.
Converted 02_core.rq_database.ipynb.
Converted 03_core.rq_marketplace.ipynb.
Converted index.ipynb.
